In [1]:
import chardet
import pandas as pd
import numpy as np
from scipy.spatial import distance
from geopy.distance import geodesic

In [2]:
def open_with_auto_sensing_encoding_types(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
        result = chardet.detect(rawdata)
        encoding_type = result['encoding']

    print(f"파일의 인코딩: {encoding_type}")
    # 감지된 인코딩으로 파일 읽기
    df = pd.read_csv(file_path, encoding=encoding_type)
    print(len(df))
    return df


def get_coord(df, x_name, y_name):
    """
    데이터프레임에서 x, y 좌표쌍을 추출하여 np.linalg.norm()을 바로 적용할 수 있는 넘파이 배열 반환.

    Parameters:
    df (pd.DataFrame): 입력 데이터프레임
    x_name (str): x좌표의 컬럼명
    y_name (str): y좌표의 컬럼명

    Returns:
    np.ndarray:
        - [[x1, y1], [x2, y2], ..., [xn, yn]] 형태의 넘파이 배열 (N, 2)
    """
    return df[[x_name, y_name]].dropna().reset_index(drop=True).to_numpy()  # NaN 제거 후 numpy 변환

def latlon_to_meters(lat1, lon1, lat2, lon2):
    """
    두 위경도 좌표 간의 거리를 미터(m) 단위로 변환하는 함수.

    Parameters:
    lat1, lon1 : 기준점 위도, 경도
    lat2, lon2 : 비교 대상 위도, 경도

    Returns:
    float: 두 좌표 간의 거리(m)
    """
    # 위도/경도 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # 평균 위도 (경도 변환 시 필요)
    avg_lat = np.radians((lat1 + lat2) / 2)

    # 위도, 경도를 미터로 변환
    lat_meters = dlat * 111320  # 위도 1도 ≈ 111.32 km
    lon_meters = dlon * (111320 * np.cos(avg_lat))  # 경도는 cos(위도) 보정 필요

    return np.sqrt(lat_meters**2 + lon_meters**2)  # 피타고라스 정리 적용

def closest_dist(target_coord, coords):
    '''
    가장 가까운 좌표의 인덱스와 거리 계산
    
    Parameters:
    target_coord (np.ndarray): 단일 좌표 (예: [x, y])
    coords (np.ndarray): 여러 개의 좌표를 가진 2D 넘파이 배열 (N, 2)
    
    Returns:
    idx (int): 가장 가까운 위치의 인덱스 번호
    dist (float): 가장 가까운 위치까지의 거리
    '''
    # 거리 계산 (위경도를 실제 거리(m)로 변환)
    distances = latlon_to_meters(coords[:, 0], coords[:, 1], target_coord[0], target_coord[1])

    # 최소 거리 인덱스 찾기
    idx = np.argmin(distances)

    # 가장 가까운 거리값 반환
    dist = distances[idx]

    return idx, dist

def calculate_min_distance(distance_df, reference_coords, data_name):
    '''
    각 행의 좌표에 대해 reference_coords에서 가장 가까운 좌표의 인덱스와 거리 계산
    
    Parameters:
    distance_df (pd.DataFrame): 좌표 데이터를 포함한 데이터프레임
    reference_coords (np.ndarray): 비교할 대상 (예: 유치원 위치 배열)
    data_name (str): 결과 컬럼명에서 사용할 데이터 이름 (예: 'kindergarden')

    Returns:
    pd.DataFrame:
        target좌표 | {data_name}_최소거리 | {data_name}_idx 형태의 데이터프레임 반환
    '''
    # 결과 저장 리스트
    results = []

    # 각 좌표에 대해 거리 계산
    for target in distance_df.to_numpy():
        idx, dist = closest_dist(target, reference_coords)  # 최소거리, 인덱스 계산
        results.append([dist, idx])

    distance_df
    # 결과 데이터프레임 생성
    result_df = pd.DataFrame(results, columns=[f"{data_name}_최소거리(m)", f"{data_name}_idx"])
    
    return result_df

def get_name(target_df, src_df, index_col, name_col):
    '''
    인덱스 값(시설 인덱스)을 실제 시설명으로 변환.

    Parameters:
    target_df (pd.DataFrame): 변환할 데이터프레임 (예: distance_results)
    src_df (pd.DataFrame): 원본 데이터프레임 (예: 시설 데이터셋)
    index_col (str): target_df에서 시설 인덱스를 저장한 컬럼명 (예: 'highschool_coord_idx')
    name_col (str): src_df에서 시설명을 저장한 컬럼명 (예: '시설명')

    Returns:
    pd.DataFrame: 인덱스 값이 시설명으로 변환된 새로운 데이터프레임
    '''
    # 인덱스를 기반으로 시설명 매핑
    index_to_name = src_df[name_col]  # src_df의 시설명 컬럼을 가져옴

    # 시설 인덱스를 시설명으로 변환 (매핑)
    target_df[f"{index_col}_시설명"] = target_df[index_col].map(index_to_name)

    return target_df


In [ ]:
def get_distance_df(
        target_distance_df,
        public_df,
        x_colname,
        y_colname,
        place_colname,
        col_name,
):
    '''
    @param:
    target_distance_df : 원본 카페 현황 파일 좌표쌍을 갖고있는 넘파이배열 (pd.DataFrame) : 예시) target_distance_df
    placefile_path : 시설정보 파일위치 (str), 예시) 'datas/public_place_datasests/고등학교현황.csv'
    x_colname : 시설파일 x축 열이름 (str), 예시) '정제WGS84경도'
    y_colname : 시설파일 y축 열이름 (str), 예시) '정제WGS84위도'
    place_colname : 시설파일 시설명 열이름 (str), 예시) '시설명'
    col_name : 시설파일 병합후 표현할 이름 (str), 'highschool'
    '''
    public_coords = get_coord(public_df, x_colname, y_colname)
    public_distance_df = calculate_min_distance(target_distance_df, public_coords, col_name)
    public_distance_df = get_name(public_distance_df, public_df, f"{col_name}_idx", place_colname)
    return public_distance_df

# 타켓 데이터파일 및 좌표 데이터프레임 생성

In [4]:
target_datafile = '/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/target/grid_data_set.csv'
target_df = open_with_auto_sensing_encoding_types(target_datafile)

파일의 인코딩: utf-8
15795


In [5]:
target_df.head(3) ## center_lat, center_lon

,Unnamed: 0,center_lat,center_lon,주차장수,경도,위도,법정동코드,주소,새주소-도로명,법정동명
0,0,37.535004,127.060210,2.0,127.060460,37.535041,10500,서울특별시 광진구 자양동 210-23,동일로2길,자양동
1,1,37.535004,127.060210,2.0,127.060450,37.535130,10500,서울특별시 광진구 자양동 210-22 대일카쎈타,동일로2길,자양동
2,2,37.533654,127.060784,1.0,127.060665,37.533727,10500,서울특별시 광진구 자양동 160-1,동일로,자양동


In [6]:
target_df = target_df[['center_lat','center_lon']].dropna().reset_index(drop=True)
target_df.head(3)

,center_lat,center_lon
0,37.535004,127.060210
1,37.535004,127.060210
2,37.533654,127.060784


In [7]:
bus_station_path = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/공공장소/광진구_버스정류장_좌표평균처리.csv"
bad_place_path = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/공공장소/유흥주점_전체.csv"
hof_place_path = "/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/공공장소/서울시광진구일반음식점인허가정보.csv"

bus_station_df = open_with_auto_sensing_encoding_types(bus_station_path)
bad_place_df = open_with_auto_sensing_encoding_types(bad_place_path)
hof_place_df = open_with_auto_sensing_encoding_types(hof_place_path)


파일의 인코딩: UTF-8-SIG
156
파일의 인코딩: UTF-8-SIG
26
파일의 인코딩: utf-8
17585


In [8]:
bus_station_df.head(3)

,역명,X좌표,Y좌표,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,...,19시승차총승객수,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수
0,CU중곡긴고랑점앞,127.093017,37.558838,0,0,0,0,0,0,0,...,47,155,34,167,28,108,11,103,0,0
1,강변역.테크노마트앞,127.094824,37.536367,0,0,0,0,0,0,0,...,3570,1677,2615,1055,2444,732,2362,567,862,192
2,강변역A,127.093775,37.536051,0,0,0,0,0,0,0,...,3749,1354,2453,1113,2415,931,2258,687,1202,322


In [9]:
bad_place_df.head(3)

,lastmodts,dtlstatenm,totepnum,wmeipcnt,bplcnm,maneipcnt,isream,jtupsoasgnno,faciltotscp,jtupsomainedf,...,wtrsplyfacilsenm,lvsenm,uptaenm,hoffepcnt,rdnwhladdr,sntuptaenm,y,ropnymd,mgtno,x
0,2024-09-10 18:17:45,폐업,0.0,0.0,한영술마시는노래방,0.0,0.0,NaN,75.33,NaN,...,상수도전용,기타,비어(바)살롱,0.0,서울특별시 광진구 광나루로 384 (화양동),비어(바)살롱,449475.467757,NaN,3040000-102-1972-06844,206330.286290
1,2021-05-11 14:01:35,폐업,NaN,0.0,팔도강산,0.0,NaN,NaN,114.10,NaN,...,상수도전용,기타,스텐드바,NaN,서울특별시 광진구 천호대로 555 (중곡동),스텐드바,450562.934994,NaN,3040000-102-1974-06848,207009.688680
2,2021-05-04 14:55:23,영업,NaN,0.0,카라노래바,0.0,NaN,NaN,89.48,NaN,...,상수도전용,기타,룸살롱,NaN,서울특별시 광진구 동일로 168 (화양동),룸살롱,449421.557205,NaN,3040000-102-1974-06852,205850.752898


In [10]:
hof_place_df.head(3)

,Unnamed: 0,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,0,3040000,3040000-101-1930-00394,1930-04-17,NaN,3,폐업,2,폐업,1995-08-02,...,NaN,NaN,NaN,NaN,NaN,N,21.00,NaN,NaN,NaN
1,1,3040000,3040000-101-1974-00270,1974-09-21,NaN,3,폐업,2,폐업,2000-12-30,...,NaN,NaN,NaN,NaN,NaN,N,35.96,NaN,NaN,NaN
2,2,3040000,3040000-101-1975-00414,1975-05-27,NaN,3,폐업,2,폐업,1995-08-09,...,NaN,NaN,NaN,NaN,NaN,N,128.72,NaN,NaN,NaN


In [11]:
hof_place_df["영업상태명"].value_counts()

폐업       13307
영업/정상     4278
Name: 영업상태명, dtype: int64

In [12]:
hof_place_df = hof_place_df[hof_place_df["영업상태명"] == "영업/정상"]

In [13]:
hof_place_df

,Unnamed: 0,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
6,6,3040000,3040000-101-1976-00655,1976-05-10,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,35.47,NaN,NaN,NaN
12,12,3040000,3040000-101-1976-05028,1976-04-27,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,62.56,NaN,NaN,NaN
16,16,3040000,3040000-101-1977-00803,1977-05-17,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,86.60,NaN,NaN,NaN
17,17,3040000,3040000-101-1977-00897,1977-07-15,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,32.34,NaN,NaN,NaN
38,38,3040000,3040000-101-1978-01439,1978-12-21,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,38.50,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17579,17579,3040000,3040000-101-2025-00086,2025-03-25,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17580,17580,3040000,3040000-101-2025-00087,2025-03-25,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17581,17581,3040000,3040000-101-2025-00088,2025-03-25,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17582,17582,3040000,3040000-101-2025-00089,2025-03-27,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
hof_place_df["업태구분명"].value_counts()

한식                 1761
호프/통닭               511
기타                  489
분식                  303
일식                  299
중국식                 247
경양식                 229
까페                   98
정종/대포집/소주방           73
식육(숯불구이)             57
외국음식전문점(인도,태국등)      55
통닭(치킨)               46
김밥(도시락)              38
횟집                   15
뷔페식                  14
패스트푸드                14
라이브카페                12
감성주점                  6
출장조리                  4
탕류(보신용)               3
패밀리레스트랑               2
냉면집                   2
Name: 업태구분명, dtype: int64

In [15]:
place_to_avoid_list = ['호프/통닭', "감성주점"]

In [16]:
place_to_avoid_list = ['호프/통닭', '감성주점']
place_to_avoid_df = hof_place_df[hof_place_df["업태구분명"].isin(place_to_avoid_list)]


In [ ]:
place_to_avoid_df.to_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/키즈카페입지분석2/data/공공장소/광진구일반호프.csv")

,Unnamed: 0,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
83,83,3040000,3040000-101-1979-02512,1979-02-08,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,37.82,NaN,NaN,NaN
306,306,3040000,3040000-101-1982-01215,1982-12-14,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,65.92,NaN,NaN,NaN
472,472,3040000,3040000-101-1983-04453,1983-04-22,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,24.50,NaN,NaN,NaN
477,477,3040000,3040000-101-1983-04594,1983-12-05,NaN,1,영업/정상,1,영업,NaN,...,NaN,NaN,NaN,NaN,NaN,N,99.80,NaN,NaN,NaN
596,596,3040000,3040000-101-1984-02768,1983-04-21,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,19.80,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17466,17466,3040000,3040000-101-2024-00441,2024-12-09,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,62.65,NaN,NaN,NaN
17522,17522,3040000,3040000-101-2025-00028,2025-02-03,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,25.00,NaN,NaN,NaN
17529,17529,3040000,3040000-101-2025-00035,2025-02-10,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,24.75,NaN,NaN,NaN
17541,17541,3040000,3040000-101-2025-00048,2025-02-21,NaN,1,영업/정상,1,영업,NaN,...,0.0,0.0,NaN,0.0,0.0,N,40.63,NaN,NaN,NaN


In [17]:
# target_datafile = 'datas/kids_cafe_dataset/키즈카페현황.csv'

# target_df = open_with_auto_sensing_encoding_types(target_datafile)
# missing_index = target_df[target_df[['영업상태명','사업장명','WGS84위도', 'WGS84경도']].isna().any(axis=1)]
# print(missing_index)

# target_df = target_df[['영업상태명','사업장명','WGS84위도', 'WGS84경도']].dropna().reset_index(drop=True)
# print(len(target_df), ':결측치 제거된 길이')

# target_coord = get_coord(target_df, 'WGS84경도', 'WGS84위도')
# target_distance_df = target_df[['WGS84경도', 'WGS84위도']].dropna()
# print(len(target_distance_df), ': 좌표 데이터셋 길이')


# 프레임워크 코드 사용예시.

위도, 경도 에대해서 dropna가 적용됐다. -> 136번째 인덱스의 행이 제거된 상태의 target_df를 기준으로한다.

인덱스 순서가 유지된다는 가정하에 작동하는 함수임으로, 다른형식으로 처리된 데이터프레임과의 병합은 concat이 아니라, 다른 함수를 써야한다,

In [18]:
# placefile_path = 'datas/public_place_datasests/고등학교현황.csv'
# public_df = open_with_auto_sensing_encoding_types(placefile_path)

# df_sample = get_distance_df(
#         target_distance_df = target_distance_df,
#         public_df = public_df,
#         x_colname = 'WGS84경도',
#         y_colname = 'WGS84위도',
#         place_colname = '시설명',
#         col_name = 'highschool',
# )
# df_sample.head(3)

# 필요한 df불러오고, 딕셔너리에 추가해서 언제든지 원하는  좌표데이터셋 추가 할 수 있게 완성

In [19]:
# # 원하는 파일을 불러오기
# highschool_datafile = 'datas/public_place_datasests/고등학교현황.csv'
# kindergarden_datafile = 'datas/public_place_datasests/유치원현황.csv'
# middleschool_datafile = 'datas/public_place_datasests/중학교현황.csv'
# elementaryschool_datafile = 'datas/public_place_datasests/초등학교현황.csv'
# private_academy_datafile = 'datas/public_place_datasests/학원현황.csv'
# public_place_datafile = 'datas/public_place_datasests/경기도공공시설개방현황.csv'
# bad_place_datafile = 'datas/public_place_datasests/유흥주점영업현황_인허가.xls'


# # df으로 정의하기
# highschool_df = open_with_auto_sensing_encoding_types(highschool_datafile)
# kindergarden_df = open_with_auto_sensing_encoding_types(kindergarden_datafile)
# middleschool_df = open_with_auto_sensing_encoding_types(middleschool_datafile)
# elementaryschool_df = open_with_auto_sensing_encoding_types(elementaryschool_datafile)
# private_academy_df = open_with_auto_sensing_encoding_types(private_academy_datafile)
# public_place_df = open_with_auto_sensing_encoding_types(public_place_datafile)
# bad_place_df = pd.read_excel(bad_place_datafile)


In [20]:
# # 시설 데이터 및 컬럼명 정보 설정
# groups = {
#     'highschool_params': [highschool_df, 'WGS84경도', 'WGS84위도', '시설명', 'highschool'],
#     'kindergarden_params': [kindergarden_df, '정제WGS84경도', '정제WGS84위도', '유치원명', 'kindergarden'],
#     'middleschool_params': [middleschool_df, 'WGS84경도', 'WGS84위도', '시설명', 'middleschool'],
#     'elementaryschool_params': [elementaryschool_df, 'WGS84경도', 'WGS84위도', '시설명', 'elementaryschool'],
#     'private_academy_params': [private_academy_df, 'WGS84경도', 'WGS84위도', '시설명', 'private_academy'],
#     'public_place_params': [public_place_df, '경도', '위도', '개방장소명', 'public_place'],
#     'bad_place_params': [bad_place_df, '경도', '위도', '사업장명', 'bad_place']
#     # 원하는 df내용 추가 수정부분
# }

In [21]:
# from geopy.distance import geodesic

# def count_nearby_facilities(kids_cafe_df, facility_df, x_colname, y_colname, place_colname, threshold_km=1.0):


#     # 초등학교 시설명, 위도, 경도 추출
#     facility_df = facility_df[[place_colname, y_colname, x_colname]].dropna()
#     # facility_location_df = facility_location_df.rename(columns={y_colname: 'WGS84위도', x_colname: 'WGS84경도'})
#     # print(facility_location_df.head(3))

#     counts = []
#     for idx, cafe in kids_cafe_df.iterrows():
#         cafe_location = (cafe['WGS84위도'], cafe['WGS84경도'])
#         count = sum(geodesic(cafe_location, (fac[y_colname], fac[x_colname])).km <= threshold_km for _, fac in facility_df.iterrows())
#         counts.append(count)
#     return np.array(counts)


# 같은 target파일에 대해서는 인덱스를 유지함으로, concat으로 최종적으로 합치기

In [22]:
# # 결과를 저장할 딕셔너리
# distance_counts = {}

# # 모든 그룹에 대해 거리 계산 수행
# for name, params in groups.items():
#     facility_df, x_colname, y_colname, place_colname, col_name = params
#     print(facility_df.head(1))
#     print()
#     print(x_colname, y_colname, place_colname, col_name)

    

#     # 반경 1km 내 시설 개수 계산
#     counts = count_nearby_facilities(target_df, facility_df, x_colname, y_colname, place_colname)

#     # 결과를 딕셔너리에 저장
#     distance_counts[name] = pd.DataFrame({
#         '사업장명': target_df['사업장명'],
#         f"{name}_반경1km_시설개수": counts
#     })

#     print(f"✅ {name} 데이터프레임 생성 완료!")

In [23]:
# # 각 시설별 거리 계산 결과 저장할 딕셔너리
# distance_results = {}

# # 모든 그룹에 대해 거리 계산 수행
# for name, params in groups.items():
#     facility_df, x_colname, y_colname, place_colname, col_name = params
#     print(f"🔄 {col_name} 거리 계산 중...")

#     # 거리 계산 수행
#     distance_results[col_name] = get_distance_df(
#         target_distance_df=target_distance_df,
#         public_df=facility_df,
#         x_colname=x_colname,
#         y_colname=y_colname,
#         place_colname=place_colname,
#         col_name=col_name
#     )

#     print(f"✅ {col_name} 거리 계산 완료!")



In [24]:
# # 모든 결과 병합
# final_df = target_df.copy()

# # 거리 계산 결과를 `final_df`에 병합
# for col_name, df in distance_results.items():
#     # 기존 DataFrame과 가로 방향으로 결합
#     final_df = pd.concat([final_df, df], axis=1)

# for col_name, df in distance_counts.items():
#     # 기존 DataFrame과 가로 방향으로 결합
#     final_df = pd.concat([final_df, df], axis=1)

# # 결과 확인
# print("병합 후 최종 데이터프레임 크기:", final_df.shape)
# final_df.head()